In [1]:
:opt no-lint

# 파서의 타입
파서의 타입을 다음과 여러 번에 결쳐 일반화할 수 있다. 
```haskell
type Parser = String -> Tree                -- 문자열을 처리해 Tree로 변환
type Parser = String -> (Tree, String)    -- 처리하고 남은 뒷부분 문자열 고려
type Parser = String -> [(Tree, String)]  -- 실패할 경우 길이 0인 빈 리스트
type Parser a = String -> [(a, String)]   -- 특정 Tree가 대신 타입 a로 일반화
type Parser a = [Char] -> [(a, [Char])]   -- 윗줄과 같음  String = [Char]
type Parser tok a = [tok] -> [(a, [tok])] -- Char 대신 토큰 타입 tok로 일반화
```
첫번째로는 가장 단순하게 파서를 문자열을 처리해 특정 추상문법나무(`Tree`)로 변환하는 것으로 이해할 수 있다.
이것은 물론 파서에 대한 올바른 설명이긴 하지만 문법분석이라는 커다란 문제를 처리하는 전체 과정에 대한 설명이다.
일반적으로 큰 문제는 작은 문제들로 쪼개서 생각할 수 있으므로 프로그래밍언어 전체의 문법도 더 작은 부분으로 나누어
더 간단한 각 부분에 대한 작은 파서들을 만드는 부분적인 문제을 해결한 다음,
이런 작은 파서를 엮어 프로그래밍언어 전체의 문법을 분석하는 파서를 구성할 수도 있을 것이다.
따라서 파서란 전체 문자열이 아닌 앞부분 일부의 문자열을 처리하여 추상문법나무(`Tree`)를 구성하고 뒷부분의 일부 문자열(`String`)이 남아있을 수도 있다는
개념으로 접근한다면 더 유연하고 일반적인 파서에 대한 설명이 될 것이다. 그러니까 두번째로는 파서를
문자열(`String`)을 받아 추상문법나무와 문자열의 순서쌍(`(Tree, String)`)을 만들어낸다고 설명할 수 있을 것이다. 
그런데 파서가 모든 문자열에 대해 성공하지는 않을 수도 있으므로 세번째로는 실패하는 경우를 빈 리스트로 표현하고
성공하는 경우를 추상문법나무와 문자열의 순서쌍이 들어 있는 비어 있지 않은 리스트를 만들어내도록 파서 타입의 정의를 일반화하였다.

그 다음부터는 특정 타입에 한정된 분석 결과(`Tree`) 및 분석 대상(`String`)이 아니라
이들을 파라메터화한 타입으로 파서를 일반화하는 과정이다. 그러니까 네번째로는 분석 결과를
특정 추상문법나무로 한정하지 않고 분석 결과의 타입(`a`)에 대해 파라메터화한 타입(`Parser a`)으로
일반할 수 있다. 마지막으로는 분석의 대상을 문자열(`String`) 즉 문자의 리스트(`[Char]`)인 경우로만
한정짓지 않고, 분석의 대상이 되는 단위를 문자(`Char`)가 아닌 일반적인 토큰 타입(`tok`)으로 추가로
파라메터화하여 파서의 타입을 `Parser tok a`로 표시하였으며, 이는 토큰열(`[tok]`)을 인자로 넘겨받아
받아 앞부분 일부를 분석한 결과(`a`)와 아직 분석하지 못한 뒷부분의 나머지 토큰열을 만들어 내려고 하는데,
혹여 실패할 수도 있으므로 `[(a, [tok])]`타입의 값을 계산해 내는 함수로써 정의된다. 

In [2]:
type Parser tok a = [tok] -> [(a, [tok])]

# 가장 간단한 파서
가장 기본적인 세 가지 파서인 `return`, `failure`, `item`을 소개하는 것으로 시작해 보자.
정확히는 `return`이란 분석 결과값(`v`)을 미리 정해주면
분석 대상을 전혀 처리하지 않고 무조건 성공하는 파서(`return v`)를 만들어내는 함수다.
반면 `failure`는 무조건 실패하는 파서다.
그리고 `item`은 주어진 토큰열로부터 맨 앞의 토큰 하나만을 분석하여
그 토큰 그대로를 분석 결과로 하는 파서이다. 그러니까 `item`은
길이 0이 아닌 모든 토큰열에 대해 성공한다.

In [3]:
return :: a -> Parser tok a -- 입력 토큰열 ts를 소비하지 않고
return v = \ts -> [(v,ts)]  -- 그냥 v를 리턴하며 성공하는 파서

failure :: Parser tok a -- 입력에 관계없이 무조건 실패하는 파서
failure = \_ -> []

In [4]:
(return 1) "abc"
failure    "abc"

[(1,"abc")]

[]

In [5]:
item :: Parser tok tok
item []     = []       -- 길이 0인 토큰열에 대해서는 실패
item (t:ts) = [(t,ts)] -- 맨 앞의 토큰 t하나만 처리해 t를 리턴 

In [6]:
item ""
item "abc"

[]

[('a',"bc")]

# 기존의 파서를 조합해 새로운 파서 만들기
기존의 파서를 조합해 새로운 파서를 만들어내는 다섯 가지 유용한 파서 컴비네이터
`(>>=)`, `(<|>)`, `sat`, `many`, `many1`에 대해 알아보자.

In [7]:
(>>=) :: Parser tok a -> (a -> Parser tok b) -> Parser tok b
p1 >>= pf = \ts -> [ (v2,ts2) | (v1,ts1) <- p1 ts,
{- 이어붙이기 sequencing -}       (v2,ts2) <- (pf v1) ts1 ]

위의 `(>>=)`는 첫째 파서와 바로 그 첫째 파서의 분석 결과에 따라 다르게 만들어질 수 있는 둘째 파서를 이어붙이는 연산으로,
첫째 파서가 분석하고 남은 토큰열을 둘째 파서로 분석한 결과를 전체 결과로 삼는다.
즉, `p1 >>= pf`는 첫번째 파서(`p1`)의 성공적인 분석 결과가 `v1`이고 아직 분석되지 않고 남아있는 토큰열 `ts1`이라 할 때,
앞선 파서의 분석 결과를 넘겨받아 따라 만들어지는 두번째 파서(`pf v1`)로 토큰열 `ts1`을 분석한 결과를 바로 천제 결과로 삼는
파서라는 말이다.

참고로 아래의 `item >>= \_ -> item`은 `item >>= (\_ -> item)`에서 하스켈 문법상 우선순위에 따라 불필요한 괄호가 생략된 표현이다.
이렇게 앞선 파서의 분석 결과를 넘겨받은 파라메터를 활용하지 않고 무시하면 앞선 결과에 무관하게 항상 똑같은 내용의 둘째 파서가 만들진다.
그리고 이 연산을 두 번 중첩해서 사용하면 세 종류의 파서를
(`item`, `item`, `return` 순서로) 차례로 엮어 이어붙인 것과 같은 파서를 만들어낼 수도 있다.

In [8]:
( item >>= \_  ->  item                            ) "abcd"
( item >>= \c1 ->  item >>= \c2 ->  return [c1,c2] ) "abcd" 

[('b',"cd")]

[("ab","cd")]

물론 다음과 같이 앞선 파서의 내용에 따라 두번째는 다른 파서(`item` 또는 `failure`)가 만들어지며 이어지도록 하는 것도 당연히 가능하다.

In [9]:
( item >>= \c -> if c=='h' then item else failure ) "hello"
( item >>= \c -> if c=='h' then item else failure ) "world"

[('e',"llo")]

[]

이미 한번 살펴본 바와 같이 `(>>=)`를 활용하며 다음과 같이 마지막에 `return`으로 이전 파서의 결과를 모아서 전체 분석 결과를 만들어내도록 조합할 수도 있다.
그리고 가독성을 위해 `(>>=)`를 쓸 때마다 줄을 바꿔가며 작성해도 좋다.

In [10]:
( item                                >>= \c1 ->
  (if c1=='h' then item else failure) >>= \c2 ->
  return [c1,c2]                                 ) "hello"
( item                                >>= \c1 ->
  (if c1=='h' then item else failure) >>= \c2 ->
  return [c1,c2]                                 ) "world"

[("he","llo")]

[]

방금 살펴본 실행 사례처럼 첫째 파서의 결과가 특정 조건을 만족할 때만 성공하는
활용 방식을 또 하나의 파서 컴비네이터로 다음과 같이 정의하자.

In [11]:
sat :: (tok -> Bool) -> Parser tok tok
sat test = item >>= \t ->          -- 토큰 하나를 읽어들여
           if test t then return t -- 조건에 맞는 경우에만 성공하고 
                     else failure  -- 그렇지 않으면 실패하는 파서

이렇게 정의된 `sat`을 활용하면 조금 전에 살펴봤던 실행 사례를 아래와 같이 더 간결하게 표현할 수 있다.

In [12]:
( sat (=='h') >>= \c1 -> item >>= \c2 -> return [c1,c2] ) "hello"
( sat (=='h') >>= \c1 -> item >>= \c2 -> return [c1,c2] ) "world"

[("he","llo")]

[]

In [13]:
(<|>) :: Parser tok a -> Parser tok a -> Parser tok a  -- 선택 choice
p1 <|> p2 = \ts -> case p1 ts of
                     []  -> p2 ts  -- 첫번째 파서가 실패하면 두번째로
                     rs1 -> rs1    -- 첫번째가 성공하면 첫번째만으로

In [14]:
item                  "ab"
(item <|> return '?') "ab"

[('a',"b")]

[('a',"b")]

In [15]:
item                  ""
(item <|> return '?') ""

[]

[('?',"")]

In [49]:
many, many1  :: Parser tok a -> Parser tok [a]  -- 상호재귀적으로 정의됨
many  p = many1 p <|> return [] -- 1회 성공 또는 무조건 0회라 치고 성공

many1 p = p      >>= \v  ->  -- 한번 성공한 다음
          many p >>= \vs ->  -- 0회 이상 성공
          return (v:vs)

`many p`와 `many1 p`는 각각 0회 및 1회 이상 주어진 파서 `p`의 분석이 연달아 성공한 결과를 차례대로 모아놓은 리스트를 전체 분석 결과로 만들어낸다.

In [17]:
( many (sat (=='h')) ) "hhhi"
( many (sat (=='h')) ) "iiii"

[("hhh","i")]

[("","iiii")]

In [18]:
( many1 (sat (=='h')) ) "hhhi"
( many1 (sat (=='h')) ) "iiii"

[("hhh","i")]

[]

# 문자열을 처리하는 파서
지금까지는 분석의 단위 대상과 분석 결과에 대해 파라메터화된 일반적인 파서에 대해 다루었다.
단지 실행 사례를 보여주기 위해 일반적인 파서를 문자열에 적용해 보았을 따름이다.
이번에는 구체적으로 분석의 대상이 문자열로 한정된 파서를 정의해 보도록 하자.

다음은 문자열에서 가장 특정 조건을 만족하는 첫 글자 하나를 처리하는 파서들이다.
하스켈 뿐 아니라 대부분의 범용 프로그래밍 언어의 표준라이브러리에서 지원할 법한,
어떤 유형(숫자, 소문자, 대문자, $\ldots$, 공백문자)의 문자인지 검사하는
조건함수(`isDigit`, `isLower`, `isUpper`, $\ldots$, `isSpace`)를
활용해 정의되어 있으며, 각 조건함수를 만족하는 첫 글자 하나만 분석한다.

In [19]:
import Data.Char ( isDigit, isLower, isUpper,
                   isAlpha, isAlphaNum, isSpace )
digit = sat isDigit
lower = sat isLower
upper = sat isUpper
letter = sat isAlpha
alphanum = sat isAlphaNum
space = sat isSpace

여기서는 `isDigit`을 활용해 정의된 `digit`에 대한 실행 사례만 아래에 나타나 있다. 나머지도 어떤 파서들인지 알아보고 시험해 보라.

In [20]:
:type isDigit
:type digit

isDigit :: Char -> Bool

digit :: Parser Char Char

In [21]:
digit "123"
digit "a23"

[('1',"23")]

[]

In [22]:
char :: Char -> Parser Char Char    -- char c는 주어진 글자 c와 첫글자가
char c = sat (==c)                  -- 일치하는 경우에만 성공하는 파서

string :: String -> Parser Char String  -- string s는 주어진 문자열 s와
string []     = return []               -- 앞부분이 일치하는 경우에만 성공
string (c:cs) = char c    >>= \_ ->
                string cs >>= \_ ->
                return (c:cs)

`char c`와 `string s`는 분석 대상 문자열의 앞부분이 주어진 글자 하나(`c`) 혹은 주어진 문자열(`s`)과 일치하는 경우에만 성공하는 파서이다.

In [23]:
(char 'h') "hello"
(char 'h') "world"

[('h',"ello")]

[]

In [24]:
(string "abc") "abcdef"
(string "abc") "ab1234"

[("abc","def")]

[]

In [25]:
nat :: Parser Char Int
nat = many1 digit >>= \s ->  -- 1개 이상의 digit(숫자)
      return (read s)        -- 문자열 s를 정수로 변환한 값으로 성공시킴

ident :: Parser Char String
ident = lower         >>= \c  ->  -- 첫글자는 소문자
        many alphanum >>= \cs ->  -- 그 뒤에는 0개 이상의 문자 또는 숫자
        return (c:cs)

spaces, spaces1 :: Parser Char ()
spaces  = many  space >>= \_ -> return ()  -- 0개 이상의 공백문자 처리
spaces1 = many1 space >>= \_ -> return ()  -- 1개 이상의 공백문자 처리

In [26]:
(many1 digit) "123def"
nat           "123def"

[("123","def")]

[(123,"def")]

In [27]:
ident "abc123defghi"
ident "abc123d  ghi"
ident "123abcd  ghi"

[("abc123defghi","")]

[("abc123d","  ghi")]

[]

In [28]:
spaces "  abc"
spaces "abc"

[((),"abc")]

[((),"abc")]

In [29]:
spaces1 "  abc"
spaces1 "abc"

[((),"abc")]

[]

\newpage
# 어휘분석(토큰화)
어휘분석(lexical analysis) 혹은 토큰화(tokenization)란 구체적 문법을 따라 작성된 문자열을 토큰열로 변환하는 과정이다.
\ref{chap:FunArithEval}장에서 다룬 FAC언어의 토큰화를 위해 우선 FAC언어의 어휘를 종류별로 분류한 토큰 데이터 타입을 다음과 같이 선언하자.

In [30]:
data Tok = KW String -- 키워드
         | ID String -- 변수 이름
         | INT Int   -- 정수
         | LP        -- (
         | RP        -- )
         | LAM       -- \
         | DOT       -- .
         | ADD       -- +
         deriving (Eq,Ord,Show)

이 장에서 지금까지 소개한 파서 컴비네이터를 활용해 FAC언어를 토큰화를 한번 직접 시도해 보라.
여러분들의 시도를 돕기 위해 FAC언어의 일부 어휘를 토큰화하는 아래와 같은 작은 토크나이저를 두 개 작성해 보았다.
`word`는 식별자를 표현하는 문자열을 분석하는 파서(`ident`)가 성공적으로 분석한 결과(`s`)가
키워드 목록으로 제시된 `if`, `then`, `else` 중 하나이면 분석 결과를 `KW s`로 그렇지 않은 일반적인 식별자이면 분석 결과를 `ID s`로 성공시킨다.
`natural`은 자연수를 표현하는 문자열을 분석하는 파서(`nat`)가 성공적으로 분석한 정수값 결과(`n`)을 토큰으로 변환한 `INT n`을 분석 결과로 하여 성공시킨다.

In [31]:
word :: Parser Char Tok
word = ident  >>= \s ->
       if s `elem` ["if","then","else"] then return (KW s)
                                        else return (ID s)
natural :: Parser Char Tok
natural = nat >>= \n -> return (INT n)

In [32]:
"if" `elem` ["if","then","else"] -- 리스트에 들어있는
"hi" `elem` ["if","then","else"] -- 원소인지 검사하기

True

False

아래의 `word`를 실행한 사례 둘을 살펴보자.
첫번째는 키워드로 분석되고 두번째는 일반적인 식별자로 분석된다.
각각의 실행 사례가 어째서 그런 분석 결과가 나오는지 설명할 수 있는지 스스로 점검해 보라.

In [33]:
word "if then else  "
word "ifthen  else  "

[(KW "if"," then else  ")]

[(ID "ifthen","  else  ")]

아래는 여러 식별자 혹은 키워드의 나열로 이루어진 연속된 단어들을 처리하기 위해 `many` 컴비네이터를 활용하려 시도했지만 기대에 못미치는 분석 결과가 나온 실행 사례이다.
왜 그냥 `word`로만 분석했을 때처럼 한 단어만 분석될까?

In [34]:
(many word) "if then else  "
(many word) "ifthen  else  "

[([KW "if"]," then else  ")]

[([ID "ifthen"],"  else  ")]

\noindent
앞의 질문에 대한 답은 바로 공백문자를 어떻게 처리해야 하는지 고려하지 않고 작성한 파서로 공백문자가 포함된 문자열에 대한 분석을 시도했기 때문이다.

많은 경우 프로그래밍언어에 나타나는 개별 어휘를 분석한 후 뒤따르는 공백문자를 건너뛰곤 한다.
이렇듯 주어진 파서로 분석에 성공한 이후 뒤따르는 공백문자를 뛰어넘는 내용까지 포함된
파서를 만들어내는 유용한 파서 컴비네이터인 `tok`를 다음과 같이 작성할 수 있다.

In [35]:
tok :: Parser Char a -> Parser Char a
tok p = p      >>= \v ->
        spaces >>= \_ ->
        return v

\noindent
이제 `tok`와 `many`를 함께 활용하면 키워드나 식별자가 연달아 나타나는 문자열의 여러 단어들을 다음과 같이 여러개의 토큰으로 이루어진 토큰열로 분석할 수 있다.

In [36]:
(many (tok word)) "if then else  "
(many (tok word)) "ifthen  else  "

[([KW "if",KW "then",KW "else"],"")]

[([ID "ifthen",KW "else"],"")]

앞서 다루었던 `<|>` 컴비네터까지 같이 활용하면 키워드나 식별자 및 정수가 여럿 나타나는 문자열을 다음과 같이 토큰열로 분석할 수 있다.
물론 아직 모든 모든 종류의 FAC언어의 어휘를 다 다루지는 않았으므로 그 이외의 어휘가 나타나면 그 이후로는 분석하지 못한 문자열이 남아있게 된다.

In [37]:
(many (tok word <|> tok natural)) "if b1 then 123 else x3  "
(many (tok word <|> tok natural)) "if b1 then 123 else 3 + (\\y. y) "

[([KW "if",ID "b1",KW "then",INT 123,KW "else",ID "x3"],"")]

[([KW "if",ID "b1",KW "then",INT 123,KW "else",INT 3],"+ (\\y. y) ")]

# 문법분석
FAC언어에 대한 어휘분석은 문자열(`[Char]`)을 분석하여 토큰열(`[Tok]`)을 만들어내는 `Parser Char [Tok]` 타입의 프로그램으로 나타낼 수 있다.
FAC언어에 대한 문법분석은 이렇게 만들어진 토큰열(`[Tok]`)을 분석하여 추상문법구조(`Expr`)를 만들어내는 `Parser Tok Expr` 타입의 프로그램으로 나타낼 수 있다. 

In [38]:
data Expr = Lit Int            -- n
          | Add Expr Expr      -- e1 + e2
          | If Expr Expr Expr  -- if e then e1 else e0
          deriving (Eq, Ord, Show)

In [39]:
aexp, aexp0, aexp1 :: Parser Tok Expr
aexp = aexp0  -- 덧셈식만 처리하는 파서

-- EBNF로는 aexp0 ::= { aexp1 "+" } aexp1
aexp0 = many ( aexp1       >>= \e ->
               sat (==ADD) >>= \_ -> 
               return e              )  >>= \es ->
        aexp1                           >>= \e' ->
        return (foldl1 Add (es ++ [e']))  -- 덧셈을 좌결합으로 처리

-- EBNF로는 aexp1 ::= lit | "(" aexp0 ")"
aexp1 = lit <|> paren aexp0

lit = sat isINT  >>= \(INT n) -> return (Lit n)
    where isINT (INT _) = True
          isINT _       = False

paren p = sat (==LP) >>= \_ ->
          p          >>= \e ->
          sat (==RP) >>= \_ ->
          return e

In [40]:
lit [INT 1]             -- 1
lit [INT 1, ADD, INT 2] -- 1 + 2

[(Lit 1,[])]

[(Lit 1,[ADD,INT 2])]

In [41]:
aexp1 [INT 2]                     -- 2
aexp1 [INT 2, ADD, INT 3]         -- 2 + 3
aexp1 [LP, INT 2, ADD, INT 3, RP] -- (2 + 3)

[(Lit 2,[])]

[(Lit 2,[ADD,INT 3])]

[(Add (Lit 2) (Lit 3),[])]

In [42]:
aexp [INT 1, ADD, INT 2]                     -- 1 + 2
aexp [INT 1, ADD, LP, INT 2, ADD, INT 3, RP] -- 1 + (2 + 3)

[(Add (Lit 1) (Lit 2),[])]

[(Add (Lit 1) (Add (Lit 2) (Lit 3)),[])]

In [43]:
expr, expr0, expr1 :: Parser Tok Expr
expr = expr0 -- Expr에 대한 파서

-- EBNF로는 expr0 ::= { expr1 "+" } (expr1 | cexpr)
expr0 = many ( expr1       >>= \e ->
               sat (==ADD) >>= \_ -> 
               return e              )  >>= \es ->
        (expr1 <|> cexpr)               >>= \e' ->
        return (foldl1 Add (es ++ [e']))

expr1 = lit <|> paren expr0

cexpr = sat (== KW "if")   >>= \_  -> expr0 >>= \e  ->
        sat (== KW "then") >>= \_  -> expr0 >>= \e1 ->
        sat (== KW "else") >>= \_  -> expr0 >>= \e0 ->
        return (If e e1 e0)

In [44]:
1 + if False then 1 else 0  -- 하스켈의 경우
1 +(if False then 1 else 0) -- 조건식 왼쪽에 오는 덧셈은 이렇게

1

1

In [45]:
if True then 1 else 0 + error "?"  -- 하스켈의 경우 조건식 오른쪽
if True then 1 else(0 + error "?") -- else 다음에 오는 덧셈은 이렇게

1

1

In [46]:
expr [INT 1,
      ADD,
      KW "if", INT 10,KW "then", INT 20, KW "else", INT 30]
expr [KW "if", INT 10,
      KW "then", INT 20,
      KW "else", INT 30, ADD, INT 1]
expr [LP, KW "if", INT 10,KW "then", INT 20, KW "else", INT 30, RP,
      ADD,
      INT 1]

[(Add (Lit 1) (If (Lit 10) (Lit 20) (Lit 30)),[])]

[(If (Lit 10) (Lit 20) (Add (Lit 30) (Lit 1)),[])]

[(Add (If (Lit 10) (Lit 20) (Lit 30)) (Lit 1),[])]

\section*{연습문제}
1. `[25]`번 셀의 `nat :: Parser Char Int`는
   `"12"`나 `"0"`처럼 자연수(0 이상의 정수)를 나타내는 문자열은 처리할 수 있지만
   `"-123"`처럼 음의 정수를 나타내는 문자열을 처리하지 못한다.
   `"12"`나 `"0"`처럼 자연수를 나타내는 문자열과 `"-123"`처럼 음의 정수를 나타내는
   문자열을 모두 처리할 수 있는, 즉 십진수 정수를 나타내는 문자열을 처리하는
   `int :: Parser Char Int`를 작성해 보라.
1. 다음 두 파서가 어떤 문자열에 대해 처리 결과가 달라지는지 실행 사례를 들고
   어째서 그러한 차이점이 발생하는지도 설명해 보라.
   - `(string "if" <|> string "then" <|> string "else" <|> ident)`
   - `(ident <|> string "if" <|> string "then" <|> string "else")`
1. `[30]`번 셀의 `Tok`는 FAC언어(\ref{chap:FunArithEval}장)의 어휘에 대한
   토큰을 나타내는 데이터 타입이다. 맨 앞에도 공백문자들이 나타나는 것을 허용하면서
   FAC언어를 나타내는 문자열을 토큰화하는
   `lexFAC :: Parser Char Tok` 함수를 작성해 보라.
1. `[38]`번 셀의 `Expr`에 람다식의 세 요소(변수, 함수요약식, 함수적용식)를 추가하여 FAC언어의 문법을 모두 나타낼 수 있도록 수정한 다음
   `[43]`번 셀의 파서도 그에 맞게 람다식의 세 요소도 처리할 수 있도록 수정해 보라.
1. `lexFAC`과 바로 위 문제에서 수정된 `expr`을 연결하여 문자열로 된 FAC언어의 소스코드를
   요약문법으로 분석하는 `parseFAC :: Parser Char Expr`을 작성하라.